In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

%pip install streetview

# get key from key.txt
api_key = open('key.txt').read()

     ------------------------------------- 381.6/381.6 kB 12.0 MB/s eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 17.1 MB/s eta 0:00:00
  Using cached typing_extensions-4.8.0-py3-none-any.whl (31 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Free

In [21]:


from streetview import search_panoramas, get_panorama_meta

location = [32.985651, -96.750306]

panos = search_panoramas(lat=location[0], lon=location[1])

metas = list(map(lambda x: get_panorama_meta(pano_id=x.pano_id, api_key=api_key), panos))

metas = [meta for meta in metas if int(meta.date[0:4]) >= 2012]
# meta = get_panorama_meta(pano_id="_R1mwpMkiqa2p0zp48EBJg", api_key=api_key)

# pano_id='_R1mwpMkiqa2p0zp48EBJg' lat=41.89820676786453 lon=12.47644220919742 heading=0.8815613985061646 pitch=89.001953125 roll=0.1744659692049026 date='2019-08'
metas

[MetaData(date='2012-07', location=Location(lat=32.98564991640238, lng=-96.75035174255531), pano_id='0HYsmn_eq5A3KanbW_wVJw'),
 MetaData(date='2012-07', location=Location(lat=32.98555930867904, lng=-96.74991852533971), pano_id='L_bhp2j3Rp6O7RBw3P8j2A'),
 MetaData(date='2012-07', location=Location(lat=32.9855607852742, lng=-96.75003205672012), pano_id='sPVxAIlj4-p5L-ItNDQfGA'),
 MetaData(date='2012-07', location=Location(lat=32.98559878928338, lng=-96.75018985750214), pano_id='5xVlIMLIfPsGYq2nbZq2cQ'),
 MetaData(date='2012-07', location=Location(lat=32.98564819059489, lng=-96.7501352152623), pano_id='FcuSERjcViRi-ac-f0UbyQ'),
 MetaData(date='2012-07', location=Location(lat=32.98564735651021, lng=-96.75002878962725), pano_id='qBC1m7SDcYhk0m_cunH13g'),
 MetaData(date='2012-07', location=Location(lat=32.98564648259163, lng=-96.74992309926196), pano_id='qxO_n2XuCKgan1Web4uv3Q'),
 MetaData(date='2012-07', location=Location(lat=32.98564573716753, lng=-96.74981977770395), pano_id='F-qpaoUC9DH9

In [ ]:
from streetview import get_panorama, get_streetview

image = get_panorama(pano_id=metas[0].pano_id, zoom=5)

# crop the black border pixels
pixels = image.load()
width, height = image.size
x0, y0 = width, height
x1, y1 = 0, 0
for i in range(width):
    i = width - i - 1
    if pixels[i, 0] != (0, 0, 0):
        x1 = i + 1
        break
    
for i in range(height):
    i = height - i - 1
    if pixels[0, i] != (0, 0, 0):
        y1 = i + 1
        break
    

image = image.crop((0, 0, x1, y1))

image.save("image.jpg", "jpeg")

# image = get_streetview(pano_id=panos[0].pano_id, api_key=api_key, heading=0)
# image2 = get_streetview(pano_id=panos[1].pano_id, api_key=api_key, heading=0)
# image3 = get_streetview(pano_id=panos[2].pano_id, api_key=api_key, heading=0)
# image.thumbnail((image.size[0]/2,image.size[1]/2))

# image2.save("image2.jpg", "jpeg")

# image = get_panorama(pano_id="_R1mwpMkiqa2p0zp48EBJg")

# image.save("image.jpg", "jpeg")


In [ ]:
def get_real_metas(_location, date):
  panos = search_panoramas(lat=_location[0], lon=_location[1])
  metas = list(map(lambda x: get_panorama_meta(pano_id=x.pano_id, api_key=api_key), panos))

  # sort the panos by by latitude north to south
  panos.sort(key=lambda x: x.lat, reverse=True)
  metas.sort(key=lambda x: x.lat, reverse=True)  

  if date != None:
    return [panos[i] for i in range(0,len(metas)) if metas[i].date == date]

  # find the newest pano in entire dataset
  year = 0
  for i in range(1,len(metas)):
    panos[i].date = metas[i].date
    if int(metas[i].date[0:4]) > year and int(metas[i].date[0:4]) != 2023:
      year = int(metas[i].date[0:4])
    
  # create a list of panos that are in the same year as the newest pano
  return [panos[i] for i in range(0,len(metas)) if int(metas[i].date[0:4]) == year]


In [ ]:
def getImageSeq(location):
  fov=120
  heading=0

  panos = search_panoramas(lat=location[0], lon=location[1])
  image = get_streetview(pano_id=panos[0].pano_id, api_key=api_key, heading=0, fov=fov)
  image2 = get_streetview(pano_id=panos[1].pano_id, api_key=api_key, heading=0, fov=fov)
  image3 = get_streetview(pano_id=panos[2].pano_id, api_key=api_key, heading=0, fov=fov)


In [ ]:
# !/usr/bin/env python
import numpy as np
from scipy import ndimage, misc
import sys, math, os
from PIL import Image


def cubemap(filename):
    SIZE = 1024
    HSIZE = SIZE / 2.0

    im = Image.open(filename)
    side_im = np.zeros((SIZE, SIZE), np.uint8)
    color_side = np.zeros((SIZE, SIZE, 3), np.uint8)
    pids = []
    for i in range(0,6):
        # Multiple process to go faster!
        pid = os.fork()
        if pid != 0:
            #  Keep track of our children
            pids.append(pid)
            continue

        #  This is numpy's way of visiting each point in an ndarray, I guess its fast...
        it = np.nditer(side_im, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            # Axis
            axA = it.multi_index[0]
            axB = it.multi_index[1]
            # Color is an axis, so we visit each point 3 times for R,G,B actually...

            # Here for each face we decide what each axis represents, x, y or z.
            z = -axA + HSIZE

            if i == 0:
                x = HSIZE
                y = -axB + HSIZE
            elif i == 1:
                x = -HSIZE
                y = axB - HSIZE
            elif i == 2:
                x = axB - HSIZE
                y = HSIZE
            elif i == 3:
                x = -axB + HSIZE
                y = -HSIZE
            elif i == 4:
                z = HSIZE
                x = axB - HSIZE
                y = axA - HSIZE
            elif i == 5:
                z = -HSIZE
                x = axB - HSIZE
                y = -axA + HSIZE

            # Now that we have x,y,z for point on plane, convert to spherical
            r = math.sqrt(float(x*x + y*y + z*z))
            theta = math.acos(float(z)/r)
            phi = -math.atan2(float(y),x)

            # Now that we have spherical, decide which pixel from the input image we want.
            ix = int((im.shape[1]-1)*phi/(2*math.pi))
            iy = int((im.shape[0]-1)*(theta)/math.pi)
            # This is faster than accessing the whole tuple! WHY???
            r = im[iy, ix, 0]
            g = im[iy, ix, 1]
            b = im[iy, ix, 2]
            color_side[axA, axB, 0] = r
            color_side[axA, axB, 1] = g
            color_side[axA, axB, 2] = b

            it.iternext()
        # Save output image using prefix, type and index info.
        pimg = Image.fromarray(color_side)
        pimg.save(os.path.join('.', "%s%d.%s"%('side_',i,'.jpg')), quality=85)

        # Children Exit here
        sys.exit(0)


    #  Thise seems to work better than waitpid(-1, 0), in that case sometimes the
    #  files still don't exist and we get an error.
    for pid in pids:
        os.waitpid(pid, 0)

In [ ]:
cubemap("image.jpg")

KeyboardInterrupt: ignored

# Test

In [ ]:
import requests

In [ ]:
meta_base = 'https://maps.googleapis.com/maps/api/streetview/metadata?'
pic_base = 'https://maps.googleapis.com/maps/api/streetview?'

# using my graduate school almar mater, GWU, as an example
location = '2121 I St NW, Washington, DC 20052'

In [ ]:
# define the params for the metadata reques
meta_params = {'key': api_key,
               'location': location}
# define the params for the picture request
pic_params = {'key': api_key,
              'location': location,
              'size': "640x640"}

In [ ]:
# obtain the metadata of the request (this is free)
meta_response = requests.get(meta_base, params=meta_params)

In [ ]:
meta_response.json()

{'copyright': '© Google',
 'date': '2022-11',
 'location': {'lat': 38.90070138128706, 'lng': -77.04766496952116},
 'pano_id': 'F2lSvBDs2NIsA1uwaE0h6Q',
 'status': 'OK'}

In [ ]:
pic_response = requests.get(pic_base, params=pic_params)

In [ ]:
with open('test.jpg', 'wb') as file:
    file.write(pic_response.content)
# remember to close the response connection to the API
pic_response.close()